In [3]:
import pandas as pd
import string
import numpy as np
import json

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow as tf

# Set random seeds for reproducibility
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

#load all the datasets 
df1 = pd.read_csv('USvideos.csv')
df2 = pd.read_csv('CAvideos.csv')
df3 = pd.read_csv('GBvideos.csv')

#load the datasets containing the category names
data1 = json.load(open('US_category_id.json'))
data2 = json.load(open('CA_category_id.json'))
data3 = json.load(open('GB_category_id.json'))


Now we need to process our data so that we can use this data to train our model for the task of title generator. Here are all the data cleaning and processing steps that we need to follow:

In [4]:
def category_extractor(data):
    i_d = [data['items'][i]['id'] for i in range(len(data['items']))]
    title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
    i_d = list(map(int, i_d))
    category = zip(i_d, title)
    category = dict(category)
    return category

#create a new category column by mapping the category names to their id
df1['category_title'] = df1['category_id'].map(category_extractor(data1))
df2['category_title'] = df2['category_id'].map(category_extractor(data2))
df3['category_title'] = df3['category_id'].map(category_extractor(data3))

#join the dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)

#drop rows based on duplicate videos
df = df.drop_duplicates('video_id')

#collect only titles of entertainment videos
#feel free to use any category of video that you want
entertainment = df[df['category_title'] == 'Entertainment']['title']
entertainment = entertainment.tolist()



#remove punctuations and convert text to lowercase
def clean_text(text):
    text = ''.join(e for e in text if e not in string.punctuation).lower()
    
    text = text.encode('utf8').decode('ascii', 'ignore')
    return text

corpus = [clean_text(e) for e in entertainment]


#### Generating Sequences

Natural language processing tasks require input data in the form of a sequence of tokens. The first step after data cleansing is to generate a sequence of n-gram tokens.

In [6]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    # Get tokens
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    # Convert to sequence of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i + 1]
            input_sequences.append(n_gram_sequence)

    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)


#### Padding the sequences

Since the sequences can be of variable length, the sequence lengths must be equal. When using neural networks, we usually feed an input into the network while waiting for output. In practice, it is more efficient to process data in batches rather than one at a time.

I will create sequences of n-gram as predictors and the following word of n-gram as label:

In [10]:
import keras.utils as ku  # Ensure this line is present at the top of your code

def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

# Call the function to generate predictors and labels
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)


Now I will use the LSTM Model to build a model for the task of Title Generator with Machine Learning:

In [13]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1  # Use the correct minus sign
    model = Sequential()
 
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
 
    # Add Hidden Layer 1 — LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
 
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))  # Use regular quotes
    model.compile(loss='categorical_crossentropy', optimizer='adam')  # Use regular quotes
 
    return model

# Create and fit the model
model = create_model(max_sequence_len, total_words)
model.fit(predictors, label, epochs=5, verbose=5)


: 

Now as we have created a function to generate titles let’s test our title generator model:

In [ ]:
print(generate_text(“spiderman”, 5, model, max_sequence_len))